<a href="https://colab.research.google.com/github/soham7707/finmodel-lab/blob/main/Individual_Bearish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance arch statsmodels matplotlib pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 7.4 MB/s eta 0:00:00


BETA


In [2]:
def recommend_stocks_beta(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 20:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']

            # --- Beta estimation ---
            X = sm.add_constant(combined['Market'])
            model = sm.OLS(combined['Stock'], X).fit()
            beta = model.params['Market']



            avg_return = np.mean(stock_returns)
            #sharpe_ratio = avg_return / predicted_vol if predicted_vol != 0 else 0

            results.append({
                'Stock': stock,
                'Beta': round(beta, 4),
                'Avg_Return': round(avg_return, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)

    # Drop rows with missing values
    df.dropna(subset=['Beta'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()


    #print(f"\n📈 Learned Weights from Regression: Beta = {beta_weight:.4f}")

    # --- Compute risk score using learned weights ---
    df['Risk_Score'] = df['Beta']
    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'Beta', 'Avg_Return', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f},  Beta: {row['Beta']}")

    return top

recommend_stocks_beta(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2022-09-01",
    end_date="2022-12-31",
    top_n=5
)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

⚠️ Not enough data for ZOMATO.NS
        Stock   Beta  Avg_Return  Risk_Score
ASIANPAINT.NS 0.3966     -0.0013      0.3966
 SUNPHARMA.NS 0.4088      0.0017      0.4088
     TITAN.NS 0.7984     -0.0001      0.7984
       ITC.NS 0.8304      0.0005      0.8304
    MARUTI.NS 0.8566     -0.0009      0.8566
BAJAJ-AUTO.NS 0.8656     -0.0014      0.8656
       TCS.NS 0.8844      0.0005      0.8844
 ICICIBANK.NS 0.9622      0.0002      0.9622
     WIPRO.NS 0.9991     -0.0005      0.9991
ULTRACEMCO.NS 1.0092      0.0006      1.0092
        LT.NS 1.0514      0.0010      1.0514
  RELIANCE.NS 1.1060     -0.0001      1.1060
  HDFCBANK.NS 1.1625      0.0012      1.1625
      INFY.NS 1.2140      0.0005      1.2140

✅ Top 5 Safest Stocks (Lowest Risk Score):
ASIANPAINT.NS → Risk Score: 0.3966,  Beta: 0.3966
SUNPHARMA.NS → Risk Score: 0.4088,  Beta: 0.4088
TITAN.NS → Risk Score: 0.7984,  Beta: 0.7984
ITC.NS → Risk Score: 0.8304,  Beta: 0.8304
MARUTI.NS → Risk Score: 0.8566,  Beta: 0.8566


,Stock,Beta,Avg_Return,Risk_Score
5,ASIANPAINT.NS,0.3966,-0.0013,0.3966
7,SUNPHARMA.NS,0.4088,0.0017,0.4088
12,TITAN.NS,0.7984,-0.0001,0.7984
8,ITC.NS,0.8304,0.0005,0.8304
6,MARUTI.NS,0.8566,-0.0009,0.8566


GARCH

In [6]:
def recommend_stocks_garch(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from arch import arch_model
    from sklearn.linear_model import LinearRegression

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 30:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']



            # --- GARCH volatility estimation ---
            garch_model = arch_model(stock_returns * 100, vol='GARCH', p=1, q=1)
            garch_result = garch_model.fit(disp='off')
            garch_volatility = np.mean(np.sqrt(garch_result.conditional_volatility)) / 100  # scale back

            avg_return = np.mean(stock_returns)
            sharpe_ratio = avg_return / garch_volatility if garch_volatility != 0 else 0

            results.append({
                'Stock': stock,
                'GARCH_Volatility': round(garch_volatility, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)

    # Drop rows with missing values
    df.dropna(subset=['GARCH_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()





    # --- Compute risk score using learned weights ---
    df['Risk_Score'] = df['GARCH_Volatility']
    df = df.sort_values(by='Risk_Score')

    print(df[['Stock','GARCH_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe Ratio: {row['Sharpe_Ratio']:.4f}, Vol: {row['GARCH_Volatility']}")

    return top

In [7]:
recommend_stocks_garch(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2022-09-01",
    end_date="2022-12-31",
    top_n=5
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

⚠️ Not enough data for ZOMATO.NS
        Stock  GARCH_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
 SUNPHARMA.NS            0.0105      0.0017        0.1584      0.0105
BAJAJ-AUTO.NS            0.0106     -0.0014       -0.1358      0.0106
 ICICIBANK.NS            0.0107      0.0002        0.0211      0.0107
  HDFCBANK.NS            0.0108      0.0012        0.1124      0.0108
       TCS.NS            0.0108      0.0005        0.0465      0.0108
       ITC.NS            0.0109      0.0005        0.0475      0.0109
        LT.NS            0.0109      0.0010        0.0957      0.0109
ASIANPAINT.NS            0.0111     -0.0013       -0.1175      0.0111
  RELIANCE.NS            0.0111     -0.0001       -0.0056      0.0111
    MARUTI.NS            0.0112     -0.0009       -0.0775      0.0112
ULTRACEMCO.NS            0.0114      0.0006        0.0506      0.0114
     WIPRO.NS            0.0116     -0.0005       -0.0458      0.0116
     TITAN.NS            0.0117     -0.0001       -0.0096

,Stock,GARCH_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
7,SUNPHARMA.NS,0.0105,0.0017,0.1584,0.0105
11,BAJAJ-AUTO.NS,0.0106,-0.0014,-0.1358,0.0106
3,ICICIBANK.NS,0.0107,0.0002,0.0211,0.0107
2,HDFCBANK.NS,0.0108,0.0012,0.1124,0.0108
1,TCS.NS,0.0108,0.0005,0.0465,0.0108


LSTM

In [8]:
def recommend_stocks_lstm(
    stocks, index_symbol, start_date, end_date,
    top_n=5
):
    import yfinance as yf
    import pandas as pd
    import numpy as np
    import statsmodels.api as sm
    from arch import arch_model
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.linear_model import LinearRegression
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense

    results = []

    # Index returns
    index_data = yf.download(index_symbol, start=start_date, end=end_date)['Close']
    index_returns = np.log(index_data / index_data.shift(1)).dropna()

    for stock in stocks:
        try:
            data = yf.download(stock, start=start_date, end=end_date)['Close']
            stock_returns = np.log(data / data.shift(1)).dropna()

            if len(stock_returns) < 30:
                print(f"⚠️ Not enough data for {stock}")
                continue

            combined = pd.concat([stock_returns, index_returns], axis=1).dropna()
            combined.columns = ['Stock', 'Market']


            # --- LSTM volatility estimation ---
            scaler = MinMaxScaler()
            scaled_returns = scaler.fit_transform(stock_returns.values.reshape(-1, 1))

            X_lstm, y_lstm = [], []
            window = 10
            for i in range(window, len(scaled_returns)):
                X_lstm.append(scaled_returns[i - window:i])
                y_lstm.append(scaled_returns[i])
            X_lstm, y_lstm = np.array(X_lstm), np.array(y_lstm)

            lstm_model = Sequential()
            lstm_model.add(LSTM(units=50, return_sequences=False, input_shape=(X_lstm.shape[1], 1)))
            lstm_model.add(Dense(1))
            lstm_model.compile(optimizer='adam', loss='mean_squared_error')
            lstm_model.fit(X_lstm, y_lstm, epochs=10, batch_size=8, verbose=0)

            predictions = lstm_model.predict(X_lstm, verbose=0)
            predicted_returns = scaler.inverse_transform(predictions)
            lstm_vol = np.std(predicted_returns)

            # --- Sharpe Ratio ---
            avg_return = np.mean(stock_returns)
            #avg_vol = np.mean([lstm_vol])
            sharpe_ratio = avg_return / lstm_vol if lstm_vol != 0 else 0

            results.append({
                'Stock': stock,
                'LSTM_Volatility': round(lstm_vol, 4),
                'Avg_Return': round(avg_return, 4),
                'Sharpe_Ratio': round(sharpe_ratio, 4)
            })

        except Exception as e:
            print(f"⚠️ Error processing {stock}: {e}")

    df = pd.DataFrame(results)
    df.dropna(subset=['LSTM_Volatility', 'Sharpe_Ratio'], inplace=True)

    if df.empty:
        print("❌ No valid stock data available after preprocessing.")
        return pd.DataFrame()


    # --- Risk Score ---
    df['Risk_Score'] = df['LSTM_Volatility']


    df = df.sort_values(by='Risk_Score')

    print(df[['Stock', 'LSTM_Volatility', 'Avg_Return', 'Sharpe_Ratio', 'Risk_Score']].to_string(index=False))

    top = df.head(top_n)
    print(f"\n✅ Top {top_n} Safest Stocks (Lowest Risk Score):")
    for _, row in top.iterrows():
        print(f"{row['Stock']} → Risk Score: {row['Risk_Score']:.4f}, Sharpe: {row['Sharpe_Ratio']:.4f}, LSTM Vol: {row['LSTM_Volatility']}")

    return top

In [9]:
recommend_stocks_lstm(
    stocks=[
        "INFY.NS", "TCS.NS", "HDFCBANK.NS", "ICICIBANK.NS", "RELIANCE.NS",
        "ASIANPAINT.NS", "MARUTI.NS", "SUNPHARMA.NS", "ITC.NS", "LT.NS",
        "ULTRACEMCO.NS", "BAJAJ-AUTO.NS", "TITAN.NS", "WIPRO.NS", "ZOMATO.NS"
    ],
    index_symbol="^NSEI",
    start_date="2022-09-01",
    end_date="2022-12-31",
    top_n=5
)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When

⚠️ Not enough data for ZOMATO.NS
        Stock  LSTM_Volatility  Avg_Return  Sharpe_Ratio  Risk_Score
     TITAN.NS           0.0014     -0.0001       -0.0780      0.0014
 SUNPHARMA.NS           0.0020      0.0017        0.8191      0.0020
ASIANPAINT.NS           0.0021     -0.0013       -0.6170      0.0021
 ICICIBANK.NS           0.0021      0.0002        0.1063      0.0021
        LT.NS           0.0021      0.0010        0.4892      0.0021
       ITC.NS           0.0022      0.0005        0.2370      0.0022
  HDFCBANK.NS           0.0022      0.0012        0.5612      0.0022
BAJAJ-AUTO.NS           0.0023     -0.0014       -0.6308      0.0023
       TCS.NS           0.0024      0.0005        0.2090      0.0024
  RELIANCE.NS           0.0024     -0.0001       -0.0262      0.0024
     WIPRO.NS           0.0027     -0.0005       -0.1990      0.0027
    MARUTI.NS           0.0029     -0.0009       -0.3047      0.0029
ULTRACEMCO.NS           0.0034      0.0006        0.1686      0.0034
 

,Stock,LSTM_Volatility,Avg_Return,Sharpe_Ratio,Risk_Score
12,TITAN.NS,0.0014,-0.0001,-0.0780,0.0014
7,SUNPHARMA.NS,0.0020,0.0017,0.8191,0.0020
5,ASIANPAINT.NS,0.0021,-0.0013,-0.6170,0.0021
3,ICICIBANK.NS,0.0021,0.0002,0.1063,0.0021
9,LT.NS,0.0021,0.0010,0.4892,0.0021
